In [1]:
# import monai
import pydicom
import pathlib
from glob import glob
import matplotlib.pyplot as plt

# Getting the list of dicom images

In [12]:
project_ID = 'project'
experiment_ID = 'LUNG1-001'
scan_ID = '0'

if mode == 'local':
    path = pathlib.Path(f'xnat-data/archive/{project_ID}/arc001/{experiment_ID}/SCANS/{scan_ID}/DICOM/').absolute()
else:
    path = pathlib.Path(f'/data/projects/{project_ID}/experiments/{experiment_ID}/SCANS/{scan_ID}/DICOM/').absolute()

files = list(path.glob('*.dcm'))
# files

# Connecting to XNAT's REST API 

In [20]:
import pyxnat
interface = pyxnat.Interface(server='https://biral.radiologics.com', user='amajdi')
# interface = pyxnat.Interface(server='http://xnat.datascience.arizona.edu', user='artin')

## setting the project

In [26]:
projects_list = list(interface.select.projects())
print('projects_list:', projects_list)


project.exists()

projects_list: [<Project Object> RADPERF `RADPERF` (protected) 0 subject 1 MR experiment (owner: beholtAdmin) (created on 2021-02-22 13:35:39.931) https://biral.radiologics.com/data/projects/RADPERF?format=html, <Project Object> RADPERF2 `RADPERF2` (protected) 0 subject  (owner: beholtAdmin) (created on 2021-02-22 13:40:10.743) https://biral.radiologics.com/data/projects/RADPERF2?format=html, <Project Object> PACSR001 `Reduction in unnecessary diagnostic imaging exam for lung cancer` (private) 2893 subjects 3908 CT experiments (owner: bobbyAdmin/ctp) (created on 2022-09-12 15:44:34.915) https://biral.radiologics.com/data/projects/PACSR001?format=html, <Project Object> prj_test `test` (public) 0 subject  (owner: amajdi) (created on 2023-01-24 16:04:05.35) https://biral.radiologics.com/data/projects/prj_test?format=html]


True

In [35]:
project_ID = 'PACSR001'
project = interface.select.project(project_ID)

ct_scans = []

for subject_ID in project.subjects().get():
    subject = project.subject(subject_ID)

    for experiment_ID in subject.experiments().get():
        experiment = interface.select.experiment(experiment_ID)

        for scan_ID in experiment.scans().get():
            if experiment.attrs.get('xnat:imageSessionData/TYPE') == 'CT':
                ct_scans.append(experiment)

    break

In [49]:
import os
import requests
from xml.etree import ElementTree as ET

# XNAT server, credentials, and output directory
xnat_url = 'https://biral.radiologics.com'
username = 'amajdi'
password = 'password!'
output_dir = '/media/XenialETF4/artin'

# Read the XML query file
with open('biral.xnat.xml', 'r') as f:
    query_xml = f.read()

# Authenticate with XNAT
session = requests.Session()
session.auth = (username, password)

# Send the XML query to XNAT's REST API
response = session.post(f'{xnat_url}/data/search', data=query_xml, headers={'Content-Type': 'application/xml'})

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML response
    root = ET.fromstring(response.text)

    # Iterate through the results and download the images
    for result in root.findall('.//xnat:MRSession', namespaces=root.nsmap):
        project_id = result.get('project')
        subject_id = result.get('subject_ID')
        experiment_id = result.get('ID')

        # Create a directory structure: project/subject/experiment
        download_path = f'{output_dir}/{project_id}/{subject_id}/{experiment_id}'
        os.makedirs(download_path, exist_ok=True)

        # Download the images in NIfTI format, if available
        response = session.get(f'{xnat_url}/data/experiments/{experiment_id}/scans/ALL/files?format=NIFTI', stream=True)

        # Check if the request was successful
        if response.status_code == 200:
            # Save the downloaded images as a ZIP file
            with open(f'{download_path}/images.zip', 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
        else:
            print(f'Error downloading images for experiment {experiment_id}: {response.status_code}')
else:
    print(f'Error querying XNAT: {response.status_code}')

# Close the session
session.close()


Error querying XNAT: 401
